# Conditionals in Union components
Conditional components modify loggers so that they only record rays with certain final states. This exercise lets you revisit the system from the previous exercise, but now armed with a more powerful tool.

First a few questions on the conditional components.

In [5]:
import mcstasscript as ms
import load_quiz

In [ ]:
quiz = load_quiz.exercise_6()

What is the purpose of a conditional component?

- A: A conditional component decides when a geometry is simulated
- B: A conditional component can affect exactly one other component
- C: A conditional component imposes a condition on logger or abs_logger components
- D: A conditional component is required to perform a simulation with Union components

In [ ]:
quiz.question_1("C")

Select the true statement about the Union_master component.

- A: There can only be one master component in an instrument
- B: A Union_master component is required to perform a simulation with Union components
- C: All Union components in an instrument file are simulated in one master component
- D: The Union_master needs to be placed on a Union geometry

In [ ]:
quiz.question_2("B")

## Revisiting exercise 5
The next cell contains the same instrument as we worked with in exercise 5.

The quiz questions for this section are not yet complete.

In [45]:
instrument = ms.McStas_instr("exercise_6", input_path="run_folder")
instrument.settings(output_path="data_folder/exercise_6")

instrument.add_component("init", "Union_init") # Necessary for current version of Union in McStas 3.X

# Set up Al material with incoherent and powder
Al_incoherent = instrument.add_component("Al_incoherent", "Incoherent_process")
Al_incoherent.sigma = "4*0.0082"
Al_incoherent.packing_factor = 1
Al_incoherent.unit_cell_volume = 66.4

Al_powder = instrument.add_component("Al_powder", "Powder_process")
Al_powder.reflections = "\"Al.laz\""

Al = instrument.add_component("Al", "Union_make_material")
Al.process_string = '"Al_incoherent,Al_powder"'
Al.my_absorption = "100*4*0.231/66.4"

# Set up YBaCuO with incoherent and single crystal
YBaCuO_incoherent = instrument.add_component("YBaCuO_incoherent", "Incoherent_process")
YBaCuO_incoherent.sigma = 2.105
YBaCuO_incoherent.unit_cell_volume = 173.28

YBaCuO_crystal = instrument.add_component("YBaCuO_crystal", "Single_crystal_process")
YBaCuO_crystal.set_parameters(
{"ax" : 3.816, "ay" : 0, "az" : 0,
 "bx" : 0, "by" : 3.886, "bz" : 0,
 "cx" : 0, "cy" : 0, "cz" : 11.677,
 "delta_d_d" : 5E-4, "mosaic" : 30, "barns" : 1,
 "reflections" : '"YBaCuO.lau"'})

YBaCuO = instrument.add_component("YBaCuO", "Union_make_material")
YBaCuO.process_string = '"YBaCuO_incoherent,YBaCuO_crystal"'
YBaCuO.my_absorption = 100*14.82/173.28

src = instrument.add_component("source", "Source_div")
src.xwidth = 0.01
src.yheight = 0.035
src.focus_aw = 0.01
src.focus_ah = 0.01
src.flux = 1E13
instrument.add_parameter("double", "wavelength", value=4.0, comment="[AA]  Mean wavelength of neutrons")
instrument.add_parameter("double", "d_wavelength", value=0.001, comment="[AA]  Wavelength spread of neutrons")
src.lambda0 = "wavelength"
src.dlambda = "d_wavelength"


# At a reference point to build the cryostat around
cryostat_center = instrument.add_component("cryostat_center", "Arm")
cryostat_center.set_AT([0, 0, 1], RELATIVE=src)

# Parameter for controlling sample rotation
A3_angle = instrument.add_parameter("A3_angle", value=30.99)

sample = instrument.add_component("sample", "Union_box")
sample.set_AT([0, 0, 0], RELATIVE=cryostat_center)
sample.set_ROTATED([0, A3_angle, 0], RELATIVE=cryostat_center)
sample.xwidth = 0.012
sample.yheight = 0.032
sample.zdepth = 0.009
sample.material_string = '"YBaCuO"'
sample.priority = 200

# Adding cryostat using the cryostat tool
orange_cryostat = ms.Cryostat("orange", instrument, max_priority=100)
orange_cryostat.set_AT(0, cryostat_center)

orange_cryostat.add_layer(inner_radius=70E-3/2, outer_radius=75E-3/2,
                          origin_to_top=160-3, top_thickness=-1E-3,
                          origin_to_bottom=40E-3, bottom_thickness=5E-3,
                          p_interact=0.2)

orange_cryostat.add_layer(inner_radius=120E-3/2, outer_radius=125E-3/2,
                          origin_to_top=190E-3, top_thickness=22E-3,
                          origin_to_bottom=80E-3, bottom_thickness=31E-3,
                          p_interact=0.2)

orange_cryostat.build(include_master=True)
master = instrument.get_component("orange_master") # grab master object

instrument.add_declare_var("double", "t_min")
instrument.add_declare_var("double", "t_max")

instrument.append_initialize("t_min = (wavelength - d_wavelength)*(1.0 - 0.18 + 0.5)/(K2V*2*PI);")
instrument.append_initialize("t_max = (wavelength + d_wavelength)*(1.0 + 0.28 + 0.5)/(K2V*2*PI);")
instrument.add_declare_var("char", "options", array=256)
instrument.append_initialize('sprintf(options,"banana, theta limits=[-180,180] bins=180, t limits=[%g %g] bins=300",t_min,t_max);')

banana_detector = instrument.add_component("banana_tof_detector", "Monitor_nD")
banana_detector.set_RELATIVE(cryostat_center)
banana_detector.xwidth = 1
banana_detector.yheight = 0.2
banana_detector.restore_neutron = 1
options = "options"
banana_detector.options = options
banana_detector.filename = '"tof_b.dat"'

banana_detector = instrument.add_component("banana_detector", "Monitor_nD")
banana_detector.set_RELATIVE(cryostat_center)
banana_detector.set_parameters(xwidth = 1.0, yheight = 0.2, filename='"banana.dat"', restore_neutron=1,
                               options='"banana, theta limits=[-180,180] bins=180"')

L_monitor = instrument.add_component("L_monitor", "L_monitor")
L_monitor.set_AT(0.3, RELATIVE=cryostat_center)
L_monitor.set_parameters(xwidth=0.3, yheight=0.3,
                         Lmin="wavelength - d_wavelength",
                         Lmax="wavelength + d_wavelength",
                         nL=300, filename='"l_mon.dat"', restore_neutron=1)

stop = instrument.add_component("stop", "Union_stop")

logger_zx = instrument.add_component("logger_zx", "Union_logger_2D_space", before=master)
logger_zx.set_parameters(D_direction_1='"z"', D1_min=-0.07, D1_max=0.07, n1=200,
                         D_direction_2='"x"', D2_min=-0.07, D2_max=0.07, n2=200,
                         filename='"logger_zx.dat"')
logger_zx.set_AT(0, cryostat_center)

logger_zy = instrument.add_component("logger_zy", "Union_logger_2D_space", before=master)
logger_zy.set_parameters(D_direction_1='"z"', D1_min=-0.07, D1_max=0.07, n1=200,
                       D_direction_2='"y"', D2_min=-0.1, D2_max=0.2, n2=200,
                       filename='"logger_zy.dat"')
logger_zy.set_AT(0, cryostat_center)

logger_2DQ = instrument.add_component("logger_2DQ", "Union_logger_2DQ", before=master)
logger_2DQ.set_parameters(Q_direction_1 = '"z"', Q1_min = -3.3, Q1_max = 3.3, n1 = 200,
                          Q_direction_2 = '"x"', Q2_min = -3.3, Q2_max = 3.3, n2 = 200,
                          filename = '"logger_2DQ_zx.dat"')

order_total_par = instrument.add_parameter("order_total", value=0, comment="Scattering order recorded by loggers, 0 for all")

logger_zx.order_total = order_total_par
logger_zy.order_total = "order_total" # Alternative allowed syntax
logger_2DQ.order_total = order_total_par

logger_2DQ = instrument.add_component("logger_2DQ_sample", "Union_logger_2DQ", before=master)
logger_2DQ.set_parameters(Q_direction_1 = '"z"', Q1_min=-3.3, Q1_max=3.3, n1=200,
                          Q_direction_2 = '"x"', Q2_min=-3.3, Q2_max=3.3, n2=200,
                          filename = '"logger_2DQ_sample_zx.dat"',
                          order_total=order_total_par, target_geometry='"sample"')

logger_2DQ = instrument.add_component("logger_2DQ_cryo", "Union_logger_2DQ", before=master)
logger_2DQ.set_parameters(Q_direction_1 = '"z"', Q1_min=-3.3, Q1_max=3.3, n1=200,
                          Q_direction_2 = '"x"', Q2_min=-3.3, Q2_max=3.3, n2=200,
                          filename = '"logger_2DQ_cryo_zx.dat"', order_total=order_total_par, 
                          target_geometry='"orange_layer_0_layer,orange_layer_1_layer"')

### Question 1
Create copies of all loggers with _con added to their name.

In [46]:
# Your code here

### Question 2
Add a conditional PSD component and a banana_tof limited to the conditional signal similar to the one in the talk on conditionals.

In [47]:
# Your code here

### Task 3
Run the simulation and investigate the origin of some of the background seen when the crystal is in the Bragg condition for a 4 Å beam with narrow wavelength width.

In [48]:
import math
wavelength = 4.0
theta = 180/3.14159*math.asin(wavelength/2.0/3.8843)
print("theta = ", theta, " at wavelength = ", wavelength)

theta =  40.062066584385505  at wavelength =  5.0


In [49]:
%matplotlib widget
import mcstasscript.jb_interface as ms_widget
ms_widget.show(instrument)

/Users/madsbertelsen/opt/anaconda3/lib/python3.8/site-packages/matplotlib/colorbar.py:316: RuntimeWarning: divide by zero encountered in log10
  ticks = ticks[(np.log10(ticks) >= np.log10(vmin) - rtol) &
/Users/madsbertelsen/opt/anaconda3/lib/python3.8/site-packages/matplotlib/colorbar.py:317: RuntimeWarning: divide by zero encountered in log10
  (np.log10(ticks) <= np.log10(vmax) + rtol)]
